<a href="https://colab.research.google.com/github/BenEmert/methylation_umap_example/blob/main/methylation_DR_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Methylation Analysis Dashboard

This notebook allows you to explore how different parameters affect **UMAP** [(arXiv)](https://arxiv.org/abs/1802.03426) and **t-SNE** embeddings using tumor methylation data from [Koelsche et al. (Nature Comms, 2020)](https://www.nature.com/articles/s41467-020-20603-4).

### How to Run
0. If viewing this at on GitHub, click the "Open in Colab" link at the top of the notebook preview.  
1.  **Start:** Go to the menu bar at the top and select **Runtime** > **Run all**.
    *   *Alternative:* Press `Shift + Enter` to run individual cells.
    2.  **Wait:** It will take **2-3 minutes** to install the necessary libraries and download the data.
    3.  **Explore:** Scroll to the bottom of the page to reach the interactive dashboard.

    ### Using the Interactive Plots
    The plots are built with Plotly and have a toolbar in the top-right corner (visible when you hover over the plot):
    *   **Zoom:** Click and drag to zoom into a specific region.
    *   **Pan:** Select the "Pan" tool (hand icon) to move around the plot.
    *   **Reset:** Double-click anywhere on the plot to reset the view.
    *   **Download:** Click the camera icon ("Download plot as a png") to save the current view.
    *   **Tooltips:** Hover over any point to see sample details (Diagnosis, ID, etc.).

    ### About the Data
    *   **Source:** Public sarcoma methylation profiles (GSE140686).
    *   **Processing:** To ensure this runs smoothly in Colab's free memory tier, we have filtered the dataset to the **top 100,000 most variable CpG sites** across 1,077 samples.
    *   **Full Pipeline:** For the complete, reproducible analysis pipeline, visit the [GitHub Repository](https://github.com/BenEmert/methylation_umap_example).

    ### Troubleshooting
    If the notebook crashes or disconnects (due to memory limits), go to **Runtime** > **Restart session** and try running it again.

In [ ]:
# @title
%%capture
# !pip install GEOparse
!pip install jupyter_bokeh

In [ ]:
# @title
import io, os, time, functools
from pathlib import Path
# import GEOparse
import requests
import gzip
import pandas as pd
import numpy as np
import umap
import colorcet as cc
import panel as pn
import plotly.express as px
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from google.colab import files, output
from typing import Dict, List, Optional, Tuple
import warnings, logging

# Suppress the  UMAP parallelism warning
warnings.filterwarnings(
    "ignore",
    message="n_jobs value 1 overridden to 1 by setting random_state",
    category=UserWarning
)

warnings.filterwarnings(
    "ignore",
    message="reference already known",
    category=UserWarning,
    module="bokeh"
)

pn.extension('plotly')
pn.extension('tabulator')
output.enable_custom_widget_manager()


In [ ]:
# @title
def download_url(url, outpath):
  filename = url.split("/")[-1]
  outfile = outpath / filename
  with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(outfile, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

    print(f"Downloaded {filename} to {outfile}")

In [ ]:
# @title
outdir = Path('./data/')
outdir.mkdir(parents=True, exist_ok=True)

meta_data_url = "https://huggingface.co/datasets/bemert/GSE140686_GPL13534/resolve/main/metadata_complete.csv"
data_url = "https://huggingface.co/datasets/bemert/GSE140686_GPL13534/resolve/main/GSE140686_Final_Betas_corrected_100k.parquet"

download_url(meta_data_url, outdir)
download_url(data_url, outdir)


Downloaded metadata_complete.csv to data/metadata_complete.csv
Downloaded GSE140686_Final_Betas_corrected_100k.parquet to data/GSE140686_Final_Betas_corrected_100k.parquet


In [ ]:
# @title
#Load Beta values and metadata
GPL13534_meta_complete = pd.read_csv("data/metadata_complete.csv", index_col = 0)
GPL13534_processed_df = pd.read_parquet("data/GSE140686_Final_Betas_corrected_100k.parquet")
GPL13534_processed_df.set_index('ID', inplace=True)

In [ ]:
# @title Methylation Dimensionality Reduction App (UMAP & t-SNE)

pn.state.clear_caches()

def prepare_data(df, selected_ids, top_n=5000, normalization="M-values",
                 variance_mode="Subset variance", global_var_order=None,
                 pca_n=50, alpha=1e-3):
    """
    Prepare the methylation data matrix for dimensionality reduction.

    Args:
        df (pd.DataFrame): DataFrame with samples as rows and features (CpG sites) as columns
        selected_ids (list): List of sample IDs to include in the analysis.
        top_n (int, optional): Number of most variable features (CpGs) to retain. Defaults to 5000.
        normalization (str, optional): Normalization method.
            Options: "M-values" (log-ratio) or "Beta values" (centered). Defaults to "M-values".
        variance_mode (str, optional): Method for calculating variance for feature selection.
            Options: "Subset variance" (re-calculate on selection) or "Global variance".
            Defaults to "Subset variance".
        global_var_order (pd.Index, optional): Pre-computed list of features sorted by global variance.
            Required if variance_mode is "Global variance". Defaults to None.
        pca_n (int, optional): Number of PCA components to keep for pre-processing.
            Defaults to 50.
        alpha (float, optional): Offset for M-value calculation to avoid log(0).
            Defaults to 1e-3.

    Returns:
        tuple: (X_reduced, used_ids)
            - X_reduced (np.ndarray): The processed data matrix (samples x pca_components).
            - used_ids (pd.Index): The indices of the samples retained in the matrix.
    """
    try:
        Xdf = df.reindex(selected_ids).dropna(how='all', axis=0)
    except KeyError:
        return None, []

    if normalization == "Beta values":
        # Center only
        scaler = StandardScaler(with_mean=True, with_std=False)
        X_vals = scaler.fit_transform(Xdf)
    elif normalization == "M-values":
        # M-value conversion with offset: log2((beta + alpha) / (1 - beta + alpha))
        X_vals = np.log2((Xdf.values + alpha) / (1.0 - Xdf.values + alpha))
    else:
        X_vals = Xdf.values

    X_norm = pd.DataFrame(X_vals, index=Xdf.index, columns=Xdf.columns)

    if variance_mode == "Global variance" and global_var_order is not None:
        # global_var_order is expected to be already sorted by variance descending
        top_features = global_var_order[:int(top_n)]
    else:
        # Calculate variance on the currently selected samples only
        var_order = X_norm.var(axis=0).sort_values(ascending=False).index
        top_features = var_order[:int(top_n)]

    Xsel = X_norm.loc[:, top_features]

    n_samples, n_feats = Xsel.shape
    max_pca = min(pca_n, n_samples, n_feats)

    if n_feats > 50 and max_pca >= 2:
        pca = PCA(n_components=int(max_pca), random_state=0)
        X_reduced = pca.fit_transform(Xsel)
    else:
        X_reduced = Xsel.values

    return X_reduced, Xsel.index


def run_embedding(X_data, algorithm='UMAP', n_neighbors=15, min_dist=0.1,
                  metric='euclidean', perplexity=30, n_iter=1000, random_state=0):
    """
    Runs the specific embedding algorithm (UMAP or t-SNE) on pre-processed data.

    Args:
        X_data (np.ndarray): The processed data matrix.
        algorithm (str, optional): The dimensionality reduction algorithm to use.
            Options: 'UMAP' or 't-SNE'. Defaults to 'UMAP'.
        n_neighbors (int, optional): UMAP parameter. Controls local vs global structure.
            Defaults to 15.
        min_dist (float, optional): UMAP parameter. Controls how tightly points are packed.
            Defaults to 0.1.
        metric (str, optional): Distance metric (e.g., 'euclidean', 'cosine', 'correlation').
            Defaults to 'euclidean'.
        perplexity (int, optional): t-SNE parameter. Controls local vs global structure.
            Defaults to 30.
        n_iter (int, optional): t-SNE parameter. Maximum number of iterations for t-SNE optimization.
            Defaults to 1000.
        random_state (int, optional): Seed for random number generation to ensure reproducibility.
            Defaults to 0.

    Returns:
        tuple: (emb, cols)
            - emb (np.ndarray): The 2D embedding coordinates (n_samples x 2).
            - cols (list): Column names for the embedding dimensions (e.g., ['UMAP1', 'UMAP2']).
    """
    if algorithm == 'UMAP':
        eff_n_neighbors = min(int(n_neighbors), X_data.shape[0] - 1)
        if eff_n_neighbors < 2: eff_n_neighbors = 2

        reducer = umap.UMAP(
            n_neighbors=eff_n_neighbors,
            n_components=2,
            min_dist=float(min_dist),
            metric=metric,
            random_state=int(random_state),
            low_memory=True
        )
        emb = reducer.fit_transform(X_data)
        cols = ['UMAP1', 'UMAP2']

    elif algorithm == 't-SNE':
        eff_perplexity = min(float(perplexity), (X_data.shape[0] - 1) / 3)
        if eff_perplexity < 1: eff_perplexity = 1

        tsne = TSNE(
            n_components=2,
            perplexity=eff_perplexity,
            max_iter=int(n_iter),
            metric=metric,
            init='pca', # PCA initialization improves stability and global structure preservation
            learning_rate='auto',
            random_state=int(random_state),
            n_jobs=-1
        )
        emb = tsne.fit_transform(X_data)
        cols = ['tSNE1', 'tSNE2']

    else:
        raise ValueError(f"Unknown algorithm: {algorithm}. Supported: 'UMAP', 't-SNE'")

    return emb, cols

def create_static_legend_html(meta):
    """
    Generates a static HTML legend with tooltips for full names.
    """
    html = ['<div style="font-family: sans-serif; font-size: 13px; padding: 10px; border: 1px solid #eee; background: #fafafa; border-radius: 5px;">']

    legend_configs = [
        ('Color_dx', 'Dx', 'Diagnosis', 'Diagnosis'),
        ('Color_Methyl', 'Methyl_Dx', 'Methylation_Class', 'Methylation Class'),
        ('Color_WHO', 'WHO_differentiation', 'WHO_differentiation', 'WHO Category')
    ]

    for col, label_col, hover_col, title in legend_configs:
        if col not in meta.columns or label_col not in meta.columns:
            continue

        actual_hover = hover_col if hover_col in meta.columns else label_col
        cols_to_select = list(set([col, label_col, actual_hover]))

        legend_data = meta[cols_to_select].drop_duplicates().dropna().sort_values(label_col)

        html.append(f'<div style="margin-bottom: 15px;"><strong>{title}</strong><div style="display: flex; flex-wrap: wrap; gap: 8px; margin-top: 5px;">')

        for _, row in legend_data.iterrows():
            c, l, h = row[col], row[label_col], row[actual_hover]
            html.append(
                f'<div style="display: flex; align-items: center; gap: 4px;" title="{h}">'
                f'<span style="width: 12px; height: 12px; background: {c}; border-radius: 2px; border: 1px solid #ccc;"></span>'
                f'<span>{l}</span></div>'
            )
        html.append('</div></div>')

    html.append('</div>')
    return ''.join(html)

def build_app(df, meta):
    """
    Constructs the interactive Panel application layout and logic.
    """
    samples = list(df.index.astype(str))
    n_samples_total, n_features = df.shape
    global_var_order = df.var(axis=0).sort_values(ascending=False).index
    uniq_dx = sorted(meta['Diagnosis'].fillna("NA").astype(str).unique())
    sample_groups = ['All'] + uniq_dx

    active_plot_toggle = pn.widgets.RadioButtonGroup(
        name='Update Plot', options=['Plot A (Left)', 'Plot B (Right)'],
        value='Plot A (Left)', button_type='primary', width=300
    )

    # Data Settings
    norm_w = pn.widgets.Select(name='Normalization', options=['M-values', 'Beta values'], value='M-values')
    variance_w = pn.widgets.Select(name='Variance Mode', options=['Subset variance', 'Global variance'], value='Subset variance')
    top_n_w = pn.widgets.IntSlider(name='Top Features (N)', start=100, end=10000, step=100, value=5000)
    pca_n_w = pn.widgets.IntSlider(name='PCA Components', start=10, end=100, step=10, value=50)

    # Algorithm Settings
    algo_w = pn.widgets.Select(name='Algorithm', options=['UMAP', 't-SNE'], value='UMAP')
    metric_w = pn.widgets.Select(name='Metric', options=['euclidean', 'cosine', 'correlation'], value='euclidean')
    random_state_w = pn.widgets.IntInput(name='Seed', value=42, width=80)
    randomize_btn = pn.widgets.Button(name='Randomize seed', width=110, align = 'end')

    # UMAP Specifics
    neighbors_w = pn.widgets.IntSlider(name='Neighbors', start=2, end=100, value=15)
    min_dist_w = pn.widgets.FloatSlider(name='Min Dist', start=0.0, end=0.99, step=0.05, value=0.1)

    # t-SNE Specifics
    perplexity_w = pn.widgets.IntSlider(name='Perplexity', start=5, end=100, value=30)
    iter_w = pn.widgets.IntInput(name='Iterations', value=1000, start=250, end=5000, step=250)

    # Create tabulator widget for selecting diagnoses/cases
    dx_counts = meta['Diagnosis'].value_counts().reset_index()
    dx_counts.columns = ['Diagnosis', 'Count']
    dx_counts = dx_counts.sort_values('Diagnosis').reset_index(drop=True)

    sample_selector = pn.widgets.Tabulator(
        dx_counts,
        width=320,
        height=300,
        show_index=False,
        selectable='checkbox',
        sizing_mode='stretch_width',
        editors={'Diagnosis': None, 'Count': None},
        formatters={'Diagnosis': {'type': 'textarea'}},
        header_filters={'Diagnosis': {'type': 'input', 'func': 'like'}},
        titles={'Diagnosis': 'Diagnosis', 'Count': 'N'},
        configuration={'virtualDom': False, 'layout': 'fitColumns'}
    )

    sample_selector.selection = list(dx_counts.index)

    # def init_selection():
    #   sample_selector.selection = list(dx_counts.index)

    # pn.state.onload(init_selection)

    selection_info = pn.pane.Markdown(
        f"**Selected:** {n_samples_total} / {n_samples_total} samples",
        width=300, styles={'font-size': '12px', 'color': '#555'}
    )

    color_col_w = pn.widgets.RadioButtonGroup(
        name='Color By',
        options=['Color_dx', 'Color_Methyl', 'Color_WHO'],
        value='Color_dx',
        button_type='default'
    )

    # Action
    run_button = pn.widgets.Button(name='Run embedding', button_type='primary', width=300)
    progress_bar = pn.indicators.Progress(name='Progress', value=0, width=300, visible=False)
    status_text = pn.pane.Markdown("", width=300)

    @pn.depends(algo_w.param.value, watch=True)
    def update_algo_visibility(algo):
        is_umap = (algo == 'UMAP')
        neighbors_w.visible = is_umap
        min_dist_w.visible = is_umap
        perplexity_w.visible = not is_umap
        iter_w.visible = not is_umap

    update_algo_visibility('UMAP')

    _cache = {}
    _last_plots = {
        'A': {'df': None, 'params': None},
        'B': {'df': None, 'params': None}
    }

    empty_fig = px.scatter(title="Click 'Run Embedding' to generate plot")
    empty_fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), paper_bgcolor="#fafafa")
    plot_pane_a = pn.pane.Plotly(empty_fig, height=600, width=600)
    plot_pane_b = pn.pane.Plotly(empty_fig, height=600, width=600)

    def get_selected_ids():
        selected_indices = sample_selector.selection

        if not selected_indices:
            return samples

        selected_dx = dx_counts.iloc[selected_indices]['Diagnosis'].tolist()

        return meta.loc[meta['Diagnosis'].isin(selected_dx), 'ID'].astype(str).tolist()

    @pn.depends(sample_selector.param.selection, watch=True)
    def update_selection_count(event):
        try:
            ids = get_selected_ids()
            n_current = len(ids)
            selection_info.object = f"**Selected:** {n_current} / {n_samples_total} samples"
        except:
            selection_info.object = "Selection error"

    def run_analysis(event):
        target = 'A' if 'Left' in active_plot_toggle.value else 'B'
        progress_bar.visible = True
        progress_bar.value = 20
        run_button.disabled = True
        status_text.object = "Preparing Data..."

        start_time = time.time()

        try:
            ids = get_selected_ids()
            if len(ids) < 3: raise ValueError("Select at least 3 samples.")

            current_params = {
                'ids': tuple(sorted(ids)),
                'top_n': top_n_w.value,
                'norm': norm_w.value,
                'var_mode': variance_w.value,
                'pca_n': pca_n_w.value,
                'algo': algo_w.value,
                'neighbors': neighbors_w.value,
                'min_dist': min_dist_w.value,
                'perplexity': perplexity_w.value,
                'iter': iter_w.value,
                'metric': metric_w.value,
                'seed': random_state_w.value,
                'color_cat': color_col_w.value
            }

            param_key = tuple(current_params.values())

            if param_key in _cache:
                status_text.object = "Loaded from Cache"
                df_emb, cols = _cache[param_key]
                progress_bar.value = 100
            else:
                progress_bar.value = 40
                X_reduced, used_ids = prepare_data(
                    df, ids, top_n_w.value, norm_w.value, variance_w.value,
                    global_var_order, pca_n_w.value
                )

                status_text.object = f"Running {algo_w.value}..."
                progress_bar.value = 60
                emb, cols = run_embedding(
                    X_reduced, algo_w.value, neighbors_w.value, min_dist_w.value,
                    metric_w.value, perplexity_w.value, iter_w.value, random_state_w.value
                )

                df_emb = pd.DataFrame(emb, index=used_ids, columns=cols)
                df_emb.index.name = 'ID'
                df_emb = df_emb.join(meta.set_index('ID'), how='left').reset_index()

                if len(_cache) > 20: _cache.pop(next(iter(_cache)))
                _cache[param_key] = (df_emb, cols)
                progress_bar.value = 100
                status_text.object = "Done"

            # Construct Title
            color_map = {
                'Color_dx': 'Diagnosis',
                'Color_Methyl': 'Methylation',
                'Color_WHO': 'WHO'
            }
            color_label = color_map.get(current_params['color_cat'], 'Custom')

            title_parts = [
                f"{current_params['algo']} ({color_label})",
                f"N={current_params['top_n']}",
                f"{current_params['norm']}"
            ]

            if current_params['algo'] == 'UMAP':
                title_parts.append(f"nn={current_params['neighbors']}")
                title_parts.append(f"min_d={current_params['min_dist']}")
            else:
                title_parts.append(f"perp={current_params['perplexity']}")
                title_parts.append(f"iter={current_params['iter']}")

            if current_params['pca_n'] > 0:
                title_parts.append(f"PCA={current_params['pca_n']}")

            title_parts.append(f"seed={current_params['seed']}")

            title_str = " | ".join(title_parts)

            c_col = current_params['color_cat']

            # Filter hover cols to reduce serialization payload
            hover_candidates = ['ID', 'Diagnosis', 'Methylation_Class', 'WHO_differentiation']
            hover_data = {c: True for c in hover_candidates if c in df_emb.columns}

            hover_data[cols[0]] = False
            hover_data[cols[1]] = False

            fig = px.scatter(
                df_emb, x=cols[0], y=cols[1],
                color=c_col, color_discrete_map='identity',
                hover_data=hover_data,
                title=title_text_clean(title_str),
                width=600, height=600
            )
            fig.update_traces(marker=dict(size=7, opacity=0.8))

            fig.update_layout(
                dragmode='pan', showlegend=False,
                plot_bgcolor='white', margin=dict(l=20, r=20, t=40, b=20),
                paper_bgcolor="white",
                xaxis=dict(showgrid=False, zeroline=False, scaleanchor='y', scaleratio=1),
                yaxis=dict(showgrid=False, zeroline=False),
                modebar=dict(
                  remove=['select2d', 'lasso2d'],
                  add=['zoom', 'pan', 'zoomIn', 'zoomOut', 'resetScale']
                )
            )

            _last_plots[target] = {'df': df_emb, 'params': current_params}

            if target == 'A':
                plot_pane_a.object = fig
            else:
                plot_pane_b.object = fig

            elapsed = time.time() - start_time
            status_text.object = f"Done ({elapsed:.1f} sec)"

        except Exception as e:
            status_text.object = f"Error: {str(e)}"
        finally:
            run_button.disabled = False
            progress_bar.visible = False

    def title_text_clean(s):
        return f"<span style='font-size:12px; color:#333'>{s}</span>"

    run_button.on_click(run_analysis)
    randomize_btn.on_click(lambda e: random_state_w.param.update(value=np.random.randint(0, 10000)))

    def get_csv(target):
        plot_data = _last_plots[target]
        df_out = plot_data['df']
        params = plot_data['params']

        if df_out is None: return io.StringIO("No data generated yet")

        sio = io.StringIO()

        # Write Parameter Headers
        sio.write(f"# Plot Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        if params:
            sio.write(f"# Algorithm: {params['algo']}\n")
            sio.write(f"# Normalization: {params['norm']}\n")
            sio.write(f"# Top Features: {params['top_n']}\n")
            sio.write(f"# PCA Components: {params['pca_n']}\n")
            sio.write(f"# Seed: {params['seed']}\n")
            if params['algo'] == 'UMAP':
                sio.write(f"# Neighbors: {params['neighbors']}\n")
                sio.write(f"# Min Dist: {params['min_dist']}\n")
            else:
                sio.write(f"# Perplexity: {params['perplexity']}\n")
                sio.write(f"# Iterations: {params['iter']}\n")

        # Write Data
        df_out.to_csv(sio, index=False)
        sio.seek(0)
        return sio

    dl_a = pn.widgets.FileDownload(callback=lambda: get_csv('A'), filename='plot_a_embedding.csv', label='Download embedding (A)', button_type='default', width=150)
    dl_b = pn.widgets.FileDownload(callback=lambda: get_csv('B'), filename='plot_b_embedding.csv', label='Download embedding (B)', button_type='default', width=150)

    # --- Final Layout Composition ---
    settings_card = pn.Card(
        norm_w, variance_w, top_n_w, pca_n_w,
        title="1. Data Processing", collapsed=False, width=320
    )

    algo_card = pn.Card(
        algo_w, metric_w,
        neighbors_w, min_dist_w, perplexity_w, iter_w,
        pn.Row(randomize_btn, random_state_w, width = 250),
        title="2. Algorithm Settings", collapsed=False, width=320
    )

    filter_content = pn.Column(
        sample_selector,
        selection_info,
        color_col_w,
        width=320,
        visible=True
    )

    filter_header_btn = pn.widgets.Button(
        name="▼ Samples & Color",
        width=320,
        button_type='default',
        styles={
            'background-color': '#f5f5f5',
            'text-align': 'left',
            'font-weight': 'bold',
            'font-size': '16px',
            'border': '1px solid #dfdfdf',
            'border-radius': '0px',
            'padding-left': '1px'
        }
    )

    def toggle_filter_section(event):
      if filter_content.visible:
          filter_content.visible = False
          filter_header_btn.name = "▶ Samples & Color"
      else:
          filter_content.visible = True
          filter_header_btn.name = "▼ Samples & Color"

    filter_header_btn.on_click(toggle_filter_section)

    # def on_app_load():
    #     sample_selector.selection = list(dx_counts.index)
    #     filter_content.visible = True
    #     filter_header_btn.name = "▼ Samples & Color"

    # pn.state.onload(on_app_load)

    # filter_card = pn.Card(
    #     sample_selector, selection_info, color_col_w,
    #     title="3. Samples & Color", collapsed=False, width=320
    # )

    sidebar = pn.Column(
        pn.pane.Markdown("### Configuration"),
        active_plot_toggle,
        settings_card,
        algo_card,
        filter_header_btn,
        filter_content,
        # filter_card,
        pn.Spacer(height=20),
        run_button,
        progress_bar,
        status_text,
        pn.Row(dl_a, dl_b),
        width=350, margin=(10, 10, 10, 10)
    )

    main_area = pn.Column(
        pn.Row(plot_pane_a, plot_pane_b),
        pn.pane.HTML(create_static_legend_html(meta), width=1200)
    )

    return pn.Row(sidebar, main_area)

app = build_app(GPL13534_processed_df, GPL13534_meta_complete)
app.servable()
app

Row
    [0] Column(margin=(10, 10, 10, 10), width=350)
        [0] Markdown(str)
        [1] RadioButtonGroup(button_type='primary', name='Update Plot', options=['Plot A (Left)', ...], value='Plot A (Left)', width=300)
        [2] Card(title='1. Data Processing', width=320)
            [0] Select(name='Normalization', options=['M-values', '...], value='M-values')
            [1] Select(name='Variance Mode', options=['Subset variance', ...], value='Subset variance')
            [2] IntSlider(end=10000, name='Top Features (N)', start=100, step=100, value=5000)
            [3] IntSlider(end=100, name='PCA Components', start=10, step=10, value=50)
        [3] Card(title='2. Algorithm Settings', width=320)
            [0] Select(name='Algorithm', options=['UMAP', 't-SNE'], value='UMAP')
            [1] Select(name='Metric', options=['euclidean', ...], value='euclidean')
            [2] IntSlider(end=100, name='Neighbors', start=2, value=15)
            [3] FloatSlider(end=0.99, name='Min Dist', step=0.05, value=0.1)
            [4] IntSlider(end=100, name='Perplexity', start=5, value=30, visible=False)
            [5] IntInput(end=5000, name='Iterations', start=250, step=250, value=1000, visible=False)
            [6] Row(width=250)
                [0] Button(align='end', name='Randomize seed', width=110)
                [1] IntInput(name='Seed', value=42, width=80)
        [4] Button(name='▼ Samples & Color', styles={'background-color': '#f5f...}, width=320)
        [5] Column(width=320)
            [0] Tabulator(editors={'Diagnosis': None, ...}, formatters={'Diagnosis': {...}, header_filters={'Diagnosis': {'type': 'in...}, height=300, min_width=320, selectable='checkbox', selection=[0, 1, 2, 3, 4, ...], show_index=False, sizing_mode='stretch_width', titles={'Diagnosis': 'Diagnosis',...}, value=              ...)
            [1] Markdown(str, styles={'font-size': '12px', ...}, width=300)
            [2] RadioButtonGroup(name='Color By', options=['Color_dx', 'Color_Methyl...], value='Color_dx')
        [6] Spacer(height=20)
        [7] Button(button_type='primary', name='Run embedding', width=300)
        [8] Progress(value=0, visible=False)
        [9] Markdown(str, width=300)
        [10] Row
            [0] FileDownload(callback=<function build_app.<local..., filename='plot_a_embedding.csv', label='Download embedding (A)', width=150)
            [1] FileDownload(callback=<function build_app.<local..., filename='plot_b_embedding.csv', label='Download embedding (B)', width=150)
    [1] Column
        [0] Row
            [0] Plotly(Figure, height=600, sizing_mode='fixed', width=600)
            [1] Plotly(Figure, height=600, sizing_mode='fixed', width=600)
        [1] HTML(str, width=1200)